In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
import seaborn as sns

In [2]:
df1=pd.read_csv('train1.csv')
df2=pd.read_csv('test1.csv')

In [3]:
def replace_nan_to_false(checker):
    if(str(checker)=='nan'):
        return False
    else:
        return True

df1['indicator_code']=df1['indicator_code'].apply(replace_nan_to_false)
df1['status']=df1['status'].apply(replace_nan_to_false)
df2['indicator_code']=df2['indicator_code'].apply(replace_nan_to_false)
df2['status']=df2['status'].apply(replace_nan_to_false)


In [4]:
y_train=df1['return']
portfolio_id=df2['portfolio_id']
df1.drop(['portfolio_id','desk_id','sold','return'], axis=1, inplace=True)
df2.drop(['portfolio_id','desk_id'], axis=1, inplace=True)


In [5]:
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

all_X = df1.append(df2)
all_X_imputed = DataFrameImputer().fit_transform(all_X)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for feature in all_X_imputed.columns:
    if all_X_imputed[feature].dtype == 'object':
        all_X_imputed[feature] = le.fit_transform(all_X_imputed[feature])


df1 = all_X_imputed[0:df1.shape[0]]
df2 = all_X_imputed[df1.shape[0]::]


                     

In [6]:

df1['new_feature']=df1['bought']-df1['sold']
df2['new_feature']=df1['bought']-df2['sold']

X_train=df1.drop(['currency','libor_rate','status','bought', 'sold'], axis=1)
X_test=df2.drop(['currency','libor_rate', 'status','bought', 'sold'], axis=1)

#test with country_code, currency, and/or, creation and sell


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:

from datetime import datetime
from dateutil.parser import parse

def to_date_str(date):
    return datetime.strptime(str(date), '%Y%m%d').strftime('%m/%d/%Y')

X_train['start_date']=X_train['start_date'].apply(to_date_str)
X_train['sell_date']=X_train['sell_date'].apply(to_date_str)
X_test['start_date']=X_test['start_date'].apply(to_date_str)
X_test['sell_date']=X_test['sell_date'].apply(to_date_str)


date_format = "%m/%d/%Y"
def to_date_obj(date_str):
    return datetime.strptime(date_str, date_format)

X_train['start_date']=X_train['start_date'].apply(to_date_obj)
X_train['sell_date']=X_train['sell_date'].apply(to_date_obj)

X_train['new_time']=X_train['start_date']-X_train['sell_date']

X_train['new_time']=(X_train['new_time'] / np.timedelta64(1, 'D')).astype(int)
X_train.drop(['start_date','sell_date'], axis=1, inplace=True)


X_test['start_date']=X_test['start_date'].apply(to_date_obj)
X_test['sell_date']=X_test['sell_date'].apply(to_date_obj)

X_test['new_time']=X_test['start_date']-X_test['sell_date']

X_test['new_time']=(X_test['new_time'] / np.timedelta64(1, 'D')).astype(int)
X_test.drop(['start_date','sell_date'], axis=1, inplace=True)


In [8]:
from sklearn.ensemble import RandomForestRegressor
rfc=RandomForestRegressor(n_estimators=500)

rfc.fit(X_train,y_train)
rfc_pred=rfc.predict(X_test)

In [10]:
rfc=RandomForestRegressor(n_estimators=500)
rfc.fit(X_train,y_train)
rfc_pred=rfc.predict(X_test)

In [11]:
prediction=pd.DataFrame(columns=['portfolio_id','return'])
prediction['return']=rfc_pred
prediction['portfolio_id']=portfolio_id

In [12]:
prediction.to_csv('bw1_example_xgboost.csv', index=False)